## Introduction
Who doesn't like to talk about the weather? It's one of the only human experience we are all sharing, whether you live in Hong-Kong or Timbuktu.
<br>
In some cultures weather discussion it's the ultimate way to survive a small talk without getting into politics and in some others it's unpolite.
<br>
Sometimes it seems the amount the interest we have in the weather is an indicator of how old be has become!
<br>
Anyway, weather is a great thing and climate change is an important discussion, so let's make some insights about it.


## Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.feature_selection import RFE, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


from datetime import datetime
import os

! pip install kaggle
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/MyDrive/deep/kaggle'

Kaggle has a dataset of the daily average temprature in major cities accross the world. It can be very useful for forecasting and global warming education.

In [3]:
! kaggle datasets download --unzip sudalairajkumar/daily-temperature-of-major-cities

  0% 0.00/12.9M [00:00<?, ?B/s]
100% 12.9M/12.9M [00:00<00:00, 292MB/s]


## It's a bit messy around here
Cleaning the dataset

In [4]:
# Load the CSV and parse dates into datetime format
def custom_date_parser(y,m,d):
 if int(y) < 1000 or int(m) not in range(1, 13) or int(d) not in range(1, 32):
   return None
 return datetime.strptime(f"{y} {m} {d}", "%Y %m %d")

df = pd.read_csv("city_temperature.csv", parse_dates={'Date': ["Year", "Month", "Day"]}, date_parser=custom_date_parser, keep_date_col=True)
df.info()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/pandas/io/parsers/base_parser.py:1055: FutureWarning: 
        Use pd.to_datetime instead.

  return generic_parser(date_parser, *date_cols)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2906327 entries, 0 to 2906326
Data columns (total 9 columns):
 #   Column          Dtype         
---  ------          -----         
 0   Date            datetime64[ns]
 1   Region          object        
 2   Country         object        
 3   State           object        
 4   City            object        
 5   Month           object        
 6   Day             object        
 7   Year            object        
 8   AvgTemperature  float64       
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 199.6+ MB


In [5]:
# clear some null values and unknown temperatures
df = df[~df['Date'].isnull()]
df = df[df["AvgTemperature"] > -99]
# change types
df['Year'] = df['Year'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Day'] = df['Day'].astype(int)
df = df.drop_duplicates()

In [6]:
# Feature function - Adding DayOfYear feature and 1 hot reprasantation
df['DayOfYear'] = df['Date'].apply(lambda x: x.dayofyear)

df['AvgCelcius'] = (df['AvgTemperature']-32)/1.8
df['AvgCelcius'] = round(df['AvgCelcius'],1)

for c in ['Region', 'Country', 'State', 'City']:
    df[f'dum_{c}'] = df[c]
    df = pd.get_dummies(df, prefix=c, columns=[c, ])

Our main feature is AvgTemperature

In [7]:
df['AvgTemperature'].describe()

count    2.806369e+06
mean     6.039275e+01
std      1.911181e+01
min     -5.000000e+01
25%      4.740000e+01
50%      6.330000e+01
75%      7.590000e+01
max      1.100000e+02
Name: AvgTemperature, dtype: float64

Except 2020 we have arounf the same amount of samples of each year

In [8]:
df.groupby('Year')['AvgTemperature'].count()

Year
1995    111422
1996    112002
1997    112222
1998    110380
1999    112678
2000    113437
2001    114547
2002    112896
2003    113557
2004    113865
2005    114045
2006    113471
2007    112452
2008    111651
2009    112249
2010    112631
2011    111129
2012    110433
2013    109498
2014    107130
2015    106079
2016    105671
2017    105691
2018    104712
2019    104047
2020     38474
Name: AvgTemperature, dtype: int64

***Ready to go***

In [9]:
df.head()

,Date,Month,Day,Year,AvgTemperature,DayOfYear,AvgCelcius,dum_Region,Region_Africa,Region_Asia,...,City_Wilkes Barre,City_Wilmington,City_Windhoek,City_Winnipeg,City_Yakima,City_Yerevan,City_Youngstown,City_Yuma,City_Zagreb,City_Zurich
0,1995-01-01,1,1,1995,64.2,1,17.9,Africa,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1995-01-02,1,2,1995,49.4,2,9.7,Africa,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1995-01-03,1,3,1995,48.8,3,9.3,Africa,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1995-01-04,1,4,1995,46.4,4,8.0,Africa,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1995-01-05,1,5,1995,47.9,5,8.8,Africa,1,0,...,0,0,0,0,0,0,0,0,0,0


## Check your country climate change!

In [10]:
country = 'Israel' #@param ['Israel', 'Italy', 'France', 'Spain', 'Algeria', 'Portugal', 'China', 'Ethiopia', 'Ukraine', 'Brazil']
israel = df[df[f'Country_{country}'] == 1]
israel.head()

,Date,Month,Day,Year,AvgTemperature,DayOfYear,AvgCelcius,dum_Region,Region_Africa,Region_Asia,...,City_Wilkes Barre,City_Wilmington,City_Windhoek,City_Winnipeg,City_Yakima,City_Yerevan,City_Youngstown,City_Yuma,City_Zagreb,City_Zurich
1014633,1995-01-01,1,1,1995,57.3,1,14.1,Middle East,0,0,...,0,0,0,0,0,0,0,0,0,0
1014634,1995-01-02,1,2,1995,56.1,2,13.4,Middle East,0,0,...,0,0,0,0,0,0,0,0,0,0
1014635,1995-01-03,1,3,1995,55.9,3,13.3,Middle East,0,0,...,0,0,0,0,0,0,0,0,0,0
1014636,1995-01-04,1,4,1995,56.9,4,13.8,Middle East,0,0,...,0,0,0,0,0,0,0,0,0,0
1014637,1995-01-05,1,5,1995,56.6,5,13.7,Middle East,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
fig = px.scatter(israel, title=f"Israel day of year / temprature relation", x='DayOfYear', y='AvgCelcius', color="Year")
fig.show()

We can check the temprature accross the years.
It makes sense there is one peak in the summer and low temprature around January.

In [12]:
fig = px.scatter(israel, title=f"Israel day of year / temprature relation", x='Month', y='AvgCelcius', color="Year")
fig.show()

One of the interesting things in this graph is that the summer time is always hot, but the beginning of the year has crazy varience.
The next bars shows how the summer is more predictable than March and April.

In [13]:
israel_std_month = israel.groupby('Month')['AvgTemperature'].std().sort_index()

In [14]:
px.bar(israel_std_month, title="Temperature std for each month in Israel", labels={'value': 'Temperature std'}).show()

### Increasing warming

Let's look at the warming across the years.
<br> First we can check mean of all the months

In [15]:
israel_years_mean = israel.groupby('Year')['AvgCelcius'].mean()
px.bar(israel_years_mean, title=f"Temperature std for each month in {country}", labels={'value': 'Temperature mean'}, log_y=True).show()

Our STD graph can help us understand a few insights:


1.   The summer in te hot countries is pretty much the same temprature accross the years.
2.   It's really diverse between Nov' and April.

Let's go deeper and look at these months

In [16]:
greatest_month_std = israel_std_month.sort_values(ascending=False).head(n=4).index
israel_years_mean = israel[israel.Month.isin(greatest_month_std)].groupby('Year')['AvgCelcius'].mean()
px.bar(israel_years_mean, title=f"Temperature std for each month in {country}", labels={'value': 'Temperature mean'}, log_y=True).show()

See how 2006 is differ from 1996~ from Jan to April.

In [17]:
fig = px.scatter(israel[israel.Year.isin([1996, 1997, 1998, 2006])], title=f"Israel day of year / temprature relation", x='DayOfYear', y='AvgCelcius', color="Year")
fig.show()

## On scale - global climate analysis

Now we'll make the same experiment on steroids.
<br>
The graph below shows the temperature across all the countries through the moths of year. Each line has an error bar which is the std.
<br>
We can see that most of the countries has large varience in the same months of the year.

In [18]:
global_months = df.groupby(['Month', 'dum_Country'])['AvgCelcius']
global_months_mean = global_months.mean().reset_index()
global_months_std = global_months.std().reset_index()

In [19]:
px.line(global_months_mean, y='AvgCelcius', x='Month', color='dum_Country', error_y=global_months_std['AvgCelcius'],
        labels={'dum_Country': 'Country', 'AvgCelcius': 'Temperature'}, title='Temperature through months in different countries').show()

**Not sure where is it - but stay away from Kuwait in the summer! ✈**

# Forecast Prediction
The graphs of the temprature looks like a parabola, let's try to model it with polinomial linear regression and see what happens.


## Model Implementation from scratch
Implement it with vander matrix and flatenning feature vectors in order to suport multiple features.

In [43]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import NoReturn, Tuple
from numpy.linalg import pinv


def split_train_test(X: pd.DataFrame, y: pd.Series, train_proportion: float = .75) \
        -> Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series]:
    """
    Randomly split given sample to a training- and testing sample
    """
    X_prime = pd.DataFrame(X).copy()
    X_prime["YY"] = y
    perm = X_prime.iloc[np.random.permutation(len(X))]
    slice_num = int(np.floor(train_proportion * len(X)))
    train = perm.iloc[:slice_num, :]
    test = perm.iloc[slice_num:, :]
    return train.drop(["YY"], 1), train["YY"], test.drop(["YY"], 1), test["YY"]


class BaseEstimator(ABC):
    def __init__(self) -> BaseEstimator:
        self.fitted_ = False

    def fit(self, X: np.ndarray, y: np.ndarray) -> BaseEstimator:
        self._fit(X, y)
        self.fitted_ = True
        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        if not self.fitted_:
            raise ValueError("Estimator must first be fitted before calling ``predict``")
        return self._predict(X)

    def loss(self, X: np.ndarray, y: np.ndarray) -> float:
        if not self.fitted_:
            raise ValueError("Estimator must first be fitted before calling ``loss``")
        return self._loss(X, y)

    @abstractmethod
    def _fit(self, X: np.ndarray, y: np.ndarray) -> NoReturn:
        raise NotImplementedError()

    @abstractmethod
    def _predict(self, X: np.ndarray) -> np.ndarray:
        raise NotImplementedError()

    @abstractmethod
    def _loss(self, X: np.ndarray, y: np.ndarray) -> float:
        raise NotImplementedError()

    def fit_predict(self, X: np.ndarray, y: np.ndarray) -> np.ndarray:
        self.fit(X, y)
        return self.predict(X)


class LinearRegression(BaseEstimator):
    def __init__(self, include_intercept: bool = True) -> LinearRegression:
        super().__init__()
        self.include_intercept_, self.coefs_ = include_intercept, None
        self._w = None

    def _fit(self, X: np.ndarray, y: np.ndarray) -> NoReturn:
        self._w = np.linalg.pinv(self._add_ones(X).T).T @ y

    def _predict(self, X: np.ndarray) -> np.ndarray:
        return self._add_ones(X) @ self._w

    def _mse(self, y_hat, y):
        return np.mean((y_hat - y)**2)

    def _add_ones(self, X: np.ndarray) -> np.ndarray:
        if self.include_intercept_:
            return np.hstack([X, np.ones((X.shape[0], 1))])
        return X

    def _loss(self, X: np.ndarray, y: np.ndarray) -> float:
        y_hat = self._predict(X)
        return self._mse(y_hat, y)


class PolynomialFitting(BaseEstimator):
    def __init__(self, k: int) -> PolynomialFitting:
        """
        Instantiate a polynomial fitting estimator
        k: Degree of polynomial to fit
        """
        super().__init__()
        self._k = k
        self._lr = LinearRegression()

    def _lin2poly(self, X: np.ndarray):
        return np.apply_along_axis(self.__transform, 0, X).reshape(X.shape[0], -1)

    def _fit(self, X: np.ndarray, y: np.ndarray) -> NoReturn:
        X_prime = self._lin2poly(X)
        return self._lr.fit(X_prime, y)

    def _predict(self, X: np.ndarray) -> np.ndarray:
        return self._lr.predict(self._lin2poly(X))

    def _loss(self, X: np.ndarray, y: np.ndarray) -> float:
        return self._lr.loss(self._lin2poly(X), y)

    def __transform(self, X: np.ndarray) -> np.ndarray:
        return np.vander(X, N=self._k)

## Hyper Parameters
There are two main questions:
1.   What is the features?
2.   What is the best dimension of the polinom?

First, Let's try to take all the features:



In [56]:
country = 'Italy' # for  example
place = df[df[f'Country_{country}'] == 1]

This function check many options and show a bar of the loss results:

In [118]:
import time

def poly_dimention_examination(tr_x, tr_y, te_x, te_y):
  start = time.time()
  l = dict()
  for d in range(1, 11):
    pf = PolynomialFitting(k=d)
    pf.fit(tr_x, tr_y) # compute weights
    lss = pf.loss(te_x, te_y) # compute loss over test set
    lss = np.round(lss, 3)
    # print(f"Polynomial fit with {d} dimensions got loss over test of {lss}")
    l[d] = lss
  
  end = time.time()
  print(f"poly_dimention_examination took {end - start} seconds.")
  
  px.bar(y=l.values(), title=f"Loss over different dimensions for polinomial fitting", x=l.keys(), labels={'y': "test loss", 'x': 'poly fit dimentions'}).show()

*Option no.1*

In [ ]:
# split train-test
categorial_y_colunms = ['AvgCelcius', 'AvgTemperature', 'Date'] + place.filter(regex='dum_').columns.tolist()
tr_x, tr_y, te_x, te_y = split_train_test(place.drop(axis=1, labels=categorial_y_colunms), place['AvgCelcius'])

poly_dimention_examination(tr_x, tr_y, te_x, te_y)

It takes too long!
<br>
Notice all the features related in a way to time and we saw the difference between the years is pretty small.
<br>
*Option no.2:* keep only the feature of the day of the year.


In [119]:
tr_x, tr_y, te_x, te_y = split_train_test(place['DayOfYear'], place['AvgCelcius'])
poly_dimention_examination(tr_x, tr_y, te_x, te_y)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



poly_dimention_examination took 0.14101004600524902 seconds.


Seems 7, 8, 9, 10 dimensions are overfitting...
<br><br>
*Option no.3:* keep both year and DayOfYear

In [120]:
tr_x, tr_y, te_x, te_y = split_train_test(place[['DayOfYear', 'Year']], place['AvgCelcius'])
poly_dimention_examination(tr_x, tr_y, te_x, te_y)

poly_dimention_examination took 0.06242799758911133 seconds.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning:

In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only



We have a winner!
<br>
A 6 dimension polinom with [Year, DayOfYear] features should be simple and rocking! 

## Forecast Prediction in your city
And show your friends how cool you are!
<br>
We can try to predict the future temperature in cities in the globe.

In [111]:
city = "Tel Aviv" #@param ["Algiers", "Bujumbura", "Cotonou", "Bangui", "Brazzaville", "Cairo", "Addis Ababa", "Libreville", "Banjul", "Conakry", "Bissau", "Abidjan", "Nairobi", "Rabat", "Antananarivo", "Nouakchott", "Lilongwe", "Maputo", "Windhoek", "Niamey", "Lagos", "Dakar", "Freetown", "Capetown", "Lome", "Tunis", "Dar Es Salaam", "Kampala", "Lusaka", "Dhaka", "Beijing", "Chengdu", "Guangzhou", "Shanghai", "Shenyang", "Hong Kong", "Bombay (Mumbai)", "Calcutta", "Chennai (Madras)", "Delhi", "Jakarta", "Osaka", "Sapporo", "Tokyo", "Almaty", "Bishkek", "Vientiane", "Kuala Lumpur", "Ulan-bator", "Rangoon", "Tel Aviv", "Jerusalem", "Your own one!"] {allow-input: true}
month = "7" #@param [1,2,3,4,5,6,7,8,9,10,11,12]
day = "1" #@param [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
year = "2022" #@param [2021, 2022, 2023, 2024, 2025]
month, day, year = int(month), int(day), int(year)
doy = datetime(year, month, day).strftime('%j')
place = df[df[f'City_{city}'] == 1]

In [112]:
pf = PolynomialFitting(k=6)
place_X = place[['DayOfYear', 'Year']]
pf.fit(place_X, place['AvgCelcius'])
celcius = pf.predict(pd.DataFrame(data=[[doy, year]], columns=place_X.columns).astype(int))
print(f"On {day}/{month}/{year} the temperature will be {np.round(celcius, 2)} celcius.")


On 1/7/2022 the temperature will be [23.77] celcius.
